<a href="https://colab.research.google.com/github/UFM-Market-Trends/UFM-Market-Trends-SDK/blob/main/web-scrapping/0_Webscrapping_Banguat.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install selenium

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 40.8 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 384.9/384.9 kB 38.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 8.7 MB/s eta 0:00:00


In [ ]:
!sudo apt install chromium-chromedriver

# https://cloud.google.com/vertex-ai/docs/workbench/managed/introduction#limitations

# https://unix.stackexchange.com/questions/8926/installing-chrome-on-linux-without-needing-to-be-root
# !mkdir ~/chrome; cd ~/chrome
# !rpm2cpio ~/Download/google-chrome-stable_current_x86_64.rpm | cpio -id
# !cd opt/google/chrome
# !ln -s /usr/lib64/libnss3.so libnss3.so.1d
# !ln -s /usr/lib64/libnssutil3.so libnssutil3.so.1d
# !ln -s /usr/lib64/libsmime3.so libsmime3.so.1d
# !ln -s /lib64/libplc4.so libplc4.so.0d
# !ln -s /lib64/libnspr4.so libnspr4.so.0d
# !ln -s /lib64/libbz2.so.1.0.6 libbz2.so.1.0
# !ln -s ~/chrome/opt/google/chrome/google-chrome ~/bin/google-chrome


In [3]:
from selenium.webdriver.common.by import By

In [4]:
from bs4 import BeautifulSoup
from selenium import webdriver

option = webdriver.ChromeOptions()
option.add_argument('--headless')
option.add_argument('--no-sandbox')
option.add_argument('--disable-dev-sh-usage')
driver = webdriver.Chrome("/usr/bin/chromedriver", # https://stackoverflow.com/questions/60686871/colab-cant-find-chromedriver-path
                          options=option)

# driver.get('https://www.imdb.com/chart/top/') # Getting page HTML through request
# soup = BeautifulSoup(driver.page_source, 'html.parser') # Parsing content using beautifulsoup. Notice driver.page_source instead of page.content
 
# links = soup.select("table tbody tr td.titleColumn a") # Selecting all of the anchors with titles
# first10 = links[:10] # Keep only the first 10 anchors
# for anchor in first10:
#     print(anchor.text) # Display the innerText of each anchor

<ipython-input-4-d9f4ca13aeb4>:8: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome("/usr/bin/chromedriver", # https://stackoverflow.com/questions/60686871/colab-cant-find-chromedriver-path


In [5]:
driver.get("https://www.banguat.gob.gt/es/page/sistema-de-cuentas-nacionales-trimestrales-ano-de-referencia-2013")
soup = BeautifulSoup(driver.page_source, 'html.parser') # Parsing content using beautifulsoup. Notice driver.page_source instead of page.content

In [6]:
# https://pythonexamples.org/python-selenium-find-element-by-xpath/
# https://www.w3schools.com/xml/xml_xpath.asp
banguat_gdp_link = driver.find_element(By.XPATH, "//table/tbody/tr[2]/td/p[3]/a")#.screenshot("test.png")
banguat_gdp_link

<selenium.webdriver.remote.webelement.WebElement (session="052804cfb010886f5f216ea3a9827b18", element="3659038b-aec5-47ee-9936-9020e6ad82c2")>

In [7]:
banguat_gdp_href = banguat_gdp_link.get_attribute("href")
banguat_gdp_href

'https://www.banguat.gob.gt/sites/default/files/banguat/cuentasnac/PIB2013/CNT_cuadros_al_2T_2022.xlsx'

In [8]:
# https://www.codingem.com/python-download-file-from-url/

import requests
URL = banguat_gdp_href
response = requests.get(URL)
file_name = "PIB-Guatemala-sistema-de-cuentas-nacionales-trimestrales-ano-de-referencia-2013-CNT_cuadros.xlsx"
open(file_name, "wb").write(response.content)

132312

In [9]:
# # https://colab.research.google.com/notebooks/snippets/gcs.ipynb
# from google.colab import auth
# auth.authenticate_user()

# project_id = 'ufm-market-trends'
# !gcloud config set project {project_id}

Updated property [core/project].


In [10]:
import datetime
today = datetime.date.today().strftime("%Y-%m-%d")

In [11]:
# https://stackoverflow.com/questions/18727347/how-to-extract-a-filename-from-a-url-and-append-a-word-to-it
import os
from urllib.parse import urlparse
url = banguat_gdp_href
a = urlparse(url)
print(a.path)                   
original_file_name = os.path.basename(a.path)
# print(os.path.basename(a.path)) 
print(original_file_name)

/sites/default/files/banguat/cuentasnac/PIB2013/CNT_cuadros_al_2T_2022.xlsx
CNT_cuadros_al_2T_2022.xlsx


In [12]:
bucket_path = "ufm-market-trends/guatemala/web-scrapping/pib-banguat"
xlsx_file = file_name

!gsutil -h "Cache-Control:public, max-age=1" cp {xlsx_file} gs://{bucket_path}
# !gsutil cp gs://{bucket_path}/{xlsx_file} gs://{bucket_path}/{'archivo'}/{today+'-'+xlsx_file}
# Provides more detail to save an archive with the original file name
!gsutil cp gs://{bucket_path}/{xlsx_file} gs://{bucket_path}/{'archivo'}/{today+'-'+original_file_name}
!gsutil cat gs://{bucket_path}/{xlsx_file}

Copying file://PIB-Guatemala-sistema-de-cuentas-nacionales-trimestrales-ano-de-referencia-2013-CNT_cuadros.xlsx [Content-Type=application/vnd.openxmlformats-officedocument.spreadsheetml.sheet]...
/ [1 files][129.2 KiB/129.2 KiB]                                                
Operation completed over 1 objects/129.2 KiB.                                    
Copying gs://ufm-market-trends/guatemala/web-scrapping/pib-banguat/PIB-Guatemala-sistema-de-cuentas-nacionales-trimestrales-ano-de-referencia-2013-CNT_cuadros.xlsx [Content-Type=application/vnd.openxmlformats-officedocument.spreadsheetml.sheet]...
/ [1 files][129.2 KiB/129.2 KiB]                                                
Operation completed over 1 objects/129.2 KiB.                                    
���/
�:맕xz�靋Iy�\�P�������Xp��J�D�BJ�54
����$�F?���J���a�*u��z�j���
&��Qq=��$�֋���֪*Fg�"����0���j0A�4,]bL�� Ը2&ˎ���1r�g�ۙ~vUre��F<��qhW~���G��{��V5ܻ�;���9�Y�^dۭ�[T6��/�5��c��6�1H�_ޒ�#G�8�IG8N;�q���� ����